# Change Feed Processor (CFP) Library

Create the `acme-webstore` database with two containers:
<br><br>
`cart`

This is our shopping cart container. It is partitioned on *cartId* to ensure changes on a given cart are always processed in the order that the occurred.
<br><br>
`lease`

This is the lease container for the CFP Library to persist bookmarks for processing the change feed. It is partitioned on *id* to keep each lease document in separate logical partitions.

In [3]:
import azure.cosmos
from azure.cosmos import PartitionKey

# Delete the database if it already exists
try:
    cosmos_client.delete_database('acme-webstore')
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code != 404:
        raise

# Create the database
database = cosmos_client.create_database('acme-webstore')
print('Created database')

# Create the cart container
cartContainer = database.create_container('cart', PartitionKey(path="/cartId"), default_ttl=-1)
print('Created cart container')

# Create the lease container
leaseContainer = database.create_container('lease', PartitionKey(path="/id"))
print('Created lease container')

Created database
Created cart container
Created lease container


In [4]:
import uuid

# Add a new cart document
firstCartDocument = {
    "id": str(uuid.uuid4()),
    "cartId": "123",
    "item": "Surface Pro",
    "quantity": 2
}
cartContainer.create_item(body=firstCartDocument)
print('Created first cart document')

Created first cart document


In [5]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.43 queryCompileTimeInMs : 0.14 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.05 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.03 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 1 retrievedDocumentSize : 306 outputDocumentCount : 1 outputDocumentSize : 143 writeOutputTimeInMs : 0 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,3d05e16e-c764-45e2-8fa1-1afab155d978,123,Surface Pro,2


**Start the CFP Library host**

Nothing happens because, by default, the CFP Library starts from the current time.
<br><br><br>
**Stop the CFP Library host**

Modify the code to start from the beginning of time:
`.WithStartTime(DateTime.MinValue.ToUniversalTime())`
<br><br><br>

**Start the CFP Library host**

Now we get all the changes from the beginning of time.
<br><br><br>
**Stop and restart CFP Library host**

Nothing happens now, because the bookmark in the lease container overrides the start time.
<br><br><br>
**Stop the CFP Library host**


In [ ]:
# Delete and recreate the lease container
database.delete_container('lease')
leaseContainer = database.create_container('lease', PartitionKey(path="/id"))
print('Deleted and recreated lease container')

**Start the CFP Library host**

Without the bookmark, processing starts again from the default or overridden start time.

In [6]:
# Add a second cart document
secondCartDocument = {
    "id": str(uuid.uuid4()),
    "cartId": "123",
    "item": "Surface Book",
    "quantity": 8
}
cartContainer.create_item(body=secondCartDocument)
print('Created second cart document')

Created second cart document


We are now capturing new documents in real-time

**Stop the CFP Library host**

In [ ]:
# Delete and recreate the lease container
database.delete_container('lease')
leaseContainer = database.create_container('lease', PartitionKey(path="/id"))
print('Deleted and recreated lease container')

**Start the CFP Library host**

Correct ordering of these changes is guaranteed at the partition key level (the cartId)

In [ ]:
# Modify the first document (change quantity from 2 to 3)
firstCartDocument['quantity'] = 3
cartContainer.replace_item(firstCartDocument['id'], firstCartDocument)
print('Modified first cart document')

We are now capturing changed documents in real-time

**Stop the CFP Library host**

In [ ]:
# Delete and recreate the lease container
database.delete_container('lease')
leaseContainer = database.create_container('lease', PartitionKey(path="/id"))
print('Deleted and recreated lease container')

**Start the CFP Library host**

We now see cart-level changes in the correct order, and without interim updates
<br><br>

In [ ]:
# Delete the second document
cartContainer.delete_item(item=secondCartDocument['id'], partition_key=secondCartDocument['cartId'])
print('Deleted second cart document')

Deleted documents are not exposed by the change feed.

The solution is to leverage `ttl` (time-to-live).

In [ ]:
# Re-add the second cart document
cartContainer.create_item(body=secondCartDocument)
print('Re-created second cart document')

View both cart documents

In [ ]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

In [ ]:
# Soft-delete the second document using TTL
secondCartDocument['ttl'] = 3
cartContainer.replace_item(secondCartDocument['id'], secondCartDocument)
print('Soft-deleted second cart document')

Change feed exposes the update with the TTL property.

Container automatically deletes the actual document 3 seconds later.

In [ ]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

**Stop the CFP Library host**

In [ ]:
# Delete and recreate the lease container
database.delete_container('lease')
leaseContainer = database.create_container('lease', PartitionKey(path="/id"))
print('Deleted and recreated lease container')

**Start the CFP Library host**

Change feed exposes only the second version of the first (updated) item, and no longer exposes the second (deleted) item.